In [1]:
import os
import sys

projdir = os.path.abspath(os.path.join(os.path.abspath(''), '..'))
sys.path.insert(1, projdir)
os.chdir(projdir)

projdir

'/home/maxjo/Projects/ds_pbl'

In [2]:
from utils.DB_connect import get_engine
from glob import glob
engine = get_engine()

import pandas as pd
from sqlalchemy import text
from sqlalchemy.types import Date, String

---
delays

In [3]:
file_pattern = os.path.join('data', 'subway_data', '노선별_지연시간_*')
files = glob(file_pattern)
print("files:", files)

df_subway_delay = pd.concat([pd.read_csv(file, encoding="euc-kr") for file in files], ignore_index=True)
df_subway_delay.drop(columns=["연번", "번호"], inplace=True)
df_subway_delay

files: ['data/subway_data/노선별_지연시간_20230831.csv', 'data/subway_data/노선별_지연시간_20241004.csv']


,지연일자,노선,지연시간대,최대지연시간
0,2023-01-01,7호선 (부천구청행),09시~18시,5분
1,2023-01-01,7호선 (온수행),09시~18시,5분
2,2023-01-01,7호선 (장암행),09시~18시,5분
3,2023-01-02,2호선 (외선),첫차~09시,20분
4,2023-01-02,7호선 (부천구청행),18시~막차,5분
...,...,...,...,...
3266,2024-08-30,4호선 (하행선),첫차~09시,10분
3267,2024-08-30,4호선 (상행선),첫차~09시,10분
3268,2024-08-30,3호선 (하행선),첫차~09시,10분
3269,2024-08-30,3호선 (상행선),첫차~09시,5분


In [4]:
df_subway_delay.to_sql(
    name='subway_delays',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '지연일자': Date(),
        '노선': String(50),
        '지연시간대': String(10),
        '최대지연시간': String(10),
    } # type: ignore
)

2025-06-18 04:10:21,096 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-06-18 04:10:21,097 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,099 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-06-18 04:10:21,100 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,101 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-06-18 04:10:21,101 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,103 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-18 04:10:21,104 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`subway_delays`
2025-06-18 04:10:21,104 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,109 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`subway_delays`
2025-06-18 04:10:21,110 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,113 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `datascience_pbl`
2025-06-18 04:10:21,114 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-1

3271

---
Sta. congestion

In [ ]:
file_pattern = os.path.join('data', 'subway_data', 'subway_sta_*')
files = glob(file_pattern)
print("files:", files)

files_df = list()
for file in files:  # 인코딩들이이 섞여있음
    df = None
    try:
        df = pd.read_csv(file, encoding="utf-8-sig", index_col=False)
        print("Reading as utf-8-sig", end=': ')
        # print(df.columns)
        
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding="euc-kr", index_col=False)
        print("Reading as euc-kr", end=': ')
        # print(df.columns)
    
    print(len(df))
    files_df.append(df)

df_station_users = pd.concat(files_df)

df_station_users.drop(columns=["등록일자"], inplace=True)
df_station_users.rename(columns={'사용일자': '일자', '노선명': '노선'}, inplace=True)

# 날짜 포멧으로 변경
df_station_users['일자'] = df_station_users['일자'].astype(str)
df_station_users['일자'] = df_station_users['일자'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:8]}')

df_station_users

files: ['data/subway_data/subway_sta_202406.csv', 'data/subway_data/subway_sta_202309.csv', 'data/subway_data/subway_sta_202404.csv', 'data/subway_data/subway_sta_202407.csv', 'data/subway_data/subway_sta_202408.csv', 'data/subway_data/subway_sta_202405.csv', 'data/subway_data/subway_sta_202311.csv', 'data/subway_data/subway_sta_202312.csv', 'data/subway_data/subway_sta_202401.csv', 'data/subway_data/subway_sta_202402.csv', 'data/subway_data/subway_sta_202403.csv', 'data/subway_data/subway_sta_202310.csv']
Reading as utf-8-sig: 18488
Reading as utf-8-sig: 18302
Reading as utf-8-sig: 18483
Reading as utf-8-sig: 19103
Reading as utf-8-sig: 19132
Reading as utf-8-sig: 19110
Reading as utf-8-sig: 18313
Reading as utf-8-sig: 18956
Reading as utf-8-sig: 19001
Reading as euc-kr: 17807
Reading as utf-8-sig: 19079
Reading as utf-8-sig: 18911


,일자,노선,역명,승차총승객수,하차총승객수
0,2024-06-01,경부선,신도림,3746,4163
1,2024-06-01,3호선,신사,24723,26535
2,2024-06-01,3호선,압구정,26592,28521
3,2024-06-01,3호선,옥수,8041,8143
4,2024-06-01,3호선,금호,6886,6157
...,...,...,...,...,...
18906,2023-10-31,중앙선,오빈,356,370
18907,2023-10-31,중앙선,양평,3385,3385
18908,2023-10-31,중앙선,원덕,405,374
18909,2023-10-31,중앙선,용문,2877,2785


In [6]:
df_station_users.to_sql(
    name='station_users',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '일자': Date(),
        '노선': String(10),
        '역명': String(50),
    } # type: ignore
)

2025-06-18 04:10:21,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-18 04:10:21,591 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`station_users`
2025-06-18 04:10:21,592 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,595 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`station_users`
2025-06-18 04:10:21,596 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,598 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `datascience_pbl`
2025-06-18 04:10:21,599 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,601 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `datascience_pbl`
2025-06-18 04:10:21,602 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,604 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `station_users`
2025-06-18 04:10:21,605 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:10:21,609 INFO sqlalchemy.engine.Engine 
DROP TABLE station_users
2025-06-18 04:10:21,610 INFO sqlalchemy.engi

224685

---
Train congestion

서울교통공사 1-8호선 30분 단위 평균 혼잡도로 30분간 지나는 열차들의 평균 혼잡도(정원대비 승차인원으로, 승차인과 좌석수가 일치할 경우를 혼잡도 34%로 산정) 입니다.(단위: %). 서울교통공사 혼잡도 데이터는 요일구분(평일, 토요일, 일요일), 호선, 역번호, 역명, 상하선구분, 30분단위 별 혼잡도 데이터로 구성되어 있습니다. (2024년 부터 분기별 데이터가 제공됩니다.) 

In [ ]:
file_pattern = os.path.join('data', 'subway_data', '지하철*')
files = glob(file_pattern)
print("files:", files)

files_df = list()
for file in files:
    df = None
    try:
        df = pd.read_csv(file, encoding="utf-8-sig", index_col=False)
        print("Reading as utf-8-sig", end=': ')
        # print(df.columns)
        
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding="euc-kr", index_col=False)
        print("Reading as euc-kr", end=': ')
        # print(df.columns)

    df.drop(columns=["연번"], inplace=True, errors='ignore')  # 연번이 있는경우도 없는 경우도 있음
    oc = list(df.columns)  # 원래 컬럼명 저장, 이후 컬럼 reorder용

    date = file[-12:-4]  # 파일명에서 일자 추출
    df['일자'] = f'{date[:4]}-{date[4:6]}-{date[6:8]}'  # YYYY-MM-DD 포멧으로 변경

    df = df[['일자'] + oc]  # 일자 컬럼을 맨 앞으로 이동

    print(len(df))
    files_df.append(df)

df_train_congestions = pd.concat(files_df)

df_train_congestions.drop(columns=["역번호", '출발역', '상하구분'], inplace=True)
df_train_congestions.rename(columns={'호선': '노선'}, inplace=True)

df_train_congestions['노선'] = df_train_congestions['노선'].astype(str)
df_train_congestions['노선'] = df_train_congestions['노선'].apply(lambda x: f'{x}호선')

df_train_congestions

files: ['data/subway_data/지하철혼잡도정보_20240930.csv', 'data/subway_data/지하철혼잡도정보_20240630.csv', 'data/subway_data/지하철혼잡도정보_20240331.csv', 'data/subway_data/지하철혼잡도정보_20241231.csv', 'data/subway_data/지하철혼잡도정보_20231231.csv']
Reading as euc-kr: 1674
Reading as euc-kr: 1668
Reading as euc-kr: 1668
Reading as euc-kr: 1662
Reading as euc-kr: 1668


,일자,요일구분,노선,5시30분,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,2024-09-30,평일,1호선,9.7,8.7,5.0,8.7,10.3,17.5,14.4,...,26.1,25.7,27.8,23.8,22.4,23.7,22.9,15.1,8.7,0.0
1,2024-09-30,평일,1호선,21.7,19.2,29.2,45.6,70.2,66.6,63.0,...,11.3,9.8,11.4,10.3,8.2,9.4,8.2,8.5,2.4,0.2
2,2024-09-30,평일,1호선,10.0,10.7,7.5,9.1,12.5,20.1,17.6,...,30.5,25.5,34.3,27.0,25.0,26.9,25.6,17.2,11.9,0.0
3,2024-09-30,평일,1호선,21.7,20.4,30.2,45.6,63.2,74.1,64.5,...,13.3,11.0,11.7,10.5,9.1,10.0,8.5,8.6,2.3,0.3
4,2024-09-30,평일,1호선,9.0,13.3,8.2,10.1,14.5,21.7,20.9,...,30.9,26.2,35.4,27.9,26.0,28.1,25.4,16.6,13.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,2023-12-31,일요일,8호선,7.1,3.2,3.8,2.5,5.4,5.0,5.4,...,3.7,3.5,4.5,3.6,3.0,2.6,2.6,1.5,0.8,0.0
1664,2023-12-31,일요일,8호선,2.0,1.8,2.9,2.7,2.4,3.2,4.6,...,5.2,5.2,6.3,5.6,3.6,4.9,2.9,2.8,0.0,0.0
1665,2023-12-31,일요일,8호선,9.8,4.3,3.4,2.7,4.6,5.3,6.0,...,2.3,1.9,3.0,1.8,1.6,1.1,1.0,0.7,0.3,0.0
1666,2023-12-31,일요일,8호선,1.2,0.8,2.5,2.1,1.9,2.1,3.2,...,5.9,5.7,6.5,6.2,3.7,5.0,3.4,2.9,0.0,0.0


In [22]:
df_train_congestions.to_sql(
    name='train_congestions',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '일자': Date(),
    } # type: ignore
)

2025-06-18 04:21:36,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-18 04:21:36,473 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`train_congestions`
2025-06-18 04:21:36,474 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:21:36,476 INFO sqlalchemy.engine.Engine DESCRIBE `datascience_pbl`.`train_congestions`
2025-06-18 04:21:36,477 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:21:36,479 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `datascience_pbl`
2025-06-18 04:21:36,480 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:21:36,481 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `datascience_pbl`
2025-06-18 04:21:36,482 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:21:36,483 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `train_congestions`
2025-06-18 04:21:36,484 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-18 04:21:36,486 INFO sqlalchemy.engine.Engine 
DROP TABLE train_congestions
2025-06-18 04:21:36,486 INFO

8340